In [174]:
# api 요청으로 log 데이터 추출(json)
# 추출된 log 데이터 저장
# requests 모듈 사용

import requests
import os
import json

url = 'http://ec2-3-37-12-122.ap-northeast-2.compute.amazonaws.com/api/data/log'
JSON_LOG_PATH_NOTE = './log_note/json_log_practice.json'

# api 요청으로 json_log 받기
def get_json_log(url):
    json_log = requests.get(url).json()
    return json_log

json_log = get_json_log(url)

# 이미 이전에 받은 데이터는 제외하고 새로운 데이터만 반환
def except_duplicate(input, log_exist):
    for i in log_exist:
        if i['recordId'] == input:
            return False
    return True

# 받은 json_log 저장
def save_json_log(filepath, log):
    # 받은 json_log 중 이미 받았던 로그(중복제외) 제외 새로운 log만 할당함
    new_log = []
    # path 를 가진 파일(로그가 저장된 파일)이 없을 경우, log 저장할 파일 생성
    if not os.path.isfile(filepath):
        with open(filepath, 'w') as f:
            json.dump(log, f, indent = 2)
        # 이 경우, 받은 json_log 전체가 새로운 log
        new_log = log
    # path 를 가진 파일 있을 경우, 해당 파일에 이어서 log 저장
    else:
        with open(filepath, 'r') as f:
            log_exist = json.load(f)
        for i in log:
            if except_duplicate(i['recordId'], log_exist):
                log_exist.append(i)
                # 이 경우, 받은 json_log 중 중복 제외하고 새로운 log 에 할당
                new_log.append(i)
        with open(filepath, 'w') as f:
            json.dump(log_exist, f, indent = 2)
    if new_log == []:
        print('No log updated')
        return
    return new_log

    

In [175]:
# pipeline을 통해 전달될 새로운 log
new_log = save_json_log(JSON_LOG_PATH_NOTE, json_log)

In [176]:
# 받은 json_log 확인
new_log

[{'recordId': 54329,
  'ArrivalTimeStamp': 1679051522.413,
  'data': 'gAAAAABkE8yXx7esOBTBwThqzzCpSkXD_C7WNjFcBkqTxYllr11pLVK46V_f9nydSB06DWNe37f5rR8V1rboNXlWUOvjSs5I3usyIqxcbzZWooyDprOHrvlgneBL-LOeQ_JOugba7i9RA6uZiBdQ-GkzD8--St7DyXvS74EQZIMnwxX1hn07wVM0AIF5sDH3hxYwNMtQpteEPSS72V26POepWSB0uQnmM3QNN4jGgb1GUPP3T50YEATx7hSpBa6_oDFKiNxQgW5nwSU-SAA4_Thj6uvPa6iBnNm0k7oMpMrbVH8lX__1IPfeLoTOdM5aLQ3x8gHD-ngdaPM78DEsBm0OfTlZEhCHdL7hkp8vIvRd2SG_HYQRCcI6yfQzVsABfum13Dn4mNJ0W2Hg13NkAHWB8TnTsXQuYlCDnbm1UR2o0DM6PXlPodaPLNWgxmMRwd-JfHTw2tryhUtgnKP2uXstiZ0Htrhi4w=='},
 {'recordId': 54330,
  'ArrivalTimeStamp': 1679051522.7,
  'data': 'gAAAAABkE8yXA4ux-WcISybTr9YoCBd-JefO1pCEoMPhmgPpN18FfMAvRzHxrUvKEONSeVYjx2BngGFFHTBPKy82bbXplNdV_koWE59mEd9nxCz7lyaOnpNc9M3N34nzeaqaW0f0uejwgwU6O_grKcnv49h7xlOiYFDCb73OHzsSOX-RNVZdewTTcEYaonpSo0D90-aUzV5XWfKLJJXjCmrEFDgo_f_wIBrmhyhDy0ZIHZzuJuRTLGMoxZRQMG7kG-KbdPB1NB3oxEjZ1-qIhgU7viIwoo8zhE08aPW3KxwiqE2r5_KqxRcEp16zNaGPTls_W1sUfXaQ8NTFvia603zHyM-NigHN2IQU_waQoQ5FZFw8y5-_jh

In [177]:
# log 데이터 갱신이 안되서, test용 임시 데이터 할당
if new_log == []:
    print('No log updated')

In [178]:
# 받은 log의 암호화된 데이터 복호화, 문자열 압축 알고리즘 적용 후 다시 암호화 및 저장
# 1. 암호화된 부분 복호화
# 2. b64uuid 모듈을 사용하여 'user_id' 64(32 / 32) 자 -> 44(22 /22) 자 압축
# 3. 'method' 를 value 별 숫자로 압축
# 4. 'url' 을 value 별 숫자로 압축
# 5. 'inDate' 다른 표기방식으로 압축
# 6. 암호화
# 7. 파일 저장(.json)

In [179]:
# 1. 암호화된 부분 복호화

from cryptography.fernet import Fernet

key = b't-jdqnDewRx9kWithdsTMS21eLrri70TpkMq2A59jX8='

def decrypt_data(log, key):
    fernet = Fernet(key)
    for i in log:
        decrypted_data = fernet.decrypt(i['data']).decode('ascii')
        i['data'] = decrypted_data
        # replace 로 ' -> " 를 변환해주는 이유는 json 모듈이 key 나 value 를 문자열로 인식하기 때문
        # json 모듈의 인식가능한 문자열은 반드시 ""로 감싸여져 있어야 한다(JavaScript 에서 유래된 모듈이다)
        # 문자열로 이루어진 'data' 의 복호화된 문자열 내부 data를 dict 로 변환
        i['data'] = json.loads(i['data'].replace("'", "\""))
    return

decrypt_data(new_log, key)

In [180]:
# 1 - 확인
new_log

[{'recordId': 54329,
  'ArrivalTimeStamp': 1679051522.413,
  'data': {'user_id': '0c98f95c5d88709f4e521a737f0af4beca0c87e6b8a15f3fe1e438c42bae7f53',
   'record_id': 54329,
   'activity': 'cart',
   'url': '/api/products/product/',
   'method': 'POST',
   'name': 'json_logger',
   'inDate': '2023-03-17T11:12:02.413Z',
   'detail': {'message': 'POST cart', 'levelname': 'INFO'}}},
 {'recordId': 54330,
  'ArrivalTimeStamp': 1679051522.7,
  'data': {'user_id': '336120f10ba42e9e8b5e5e725c0f491bca7cd1968c318f43d8ec7c29aa1f990a',
   'record_id': 54330,
   'activity': 'purchase',
   'url': '/api/products/product/',
   'method': 'POST',
   'name': 'json_logger',
   'inDate': '2023-03-17T11:12:02.700Z',
   'detail': {'message': 'POST purchase', 'levelname': 'INFO'}}},
 {'recordId': 54331,
  'ArrivalTimeStamp': 1679051522.988,
  'data': {'user_id': 'b44afbe7df2f54ff3527cedd3476d6d2454cc00e2f85aae25fb70110e23b3f3e',
   'record_id': 54331,
   'activity': 'view',
   'url': '/api/products/product/',
 

In [181]:
# 2. b64uuid 모듈을 사용하여 'user_id' 64(32 / 32) 자 -> 44(22 /22) 자 압축
from b64uuid import B64UUID

def to_b64uuid(log):
    for i in log:
        i['data']['user_id'] = B64UUID(i['data']['user_id'][:32]).string + B64UUID(i['data']['user_id'][32:]).string
    return

to_b64uuid(new_log)


In [182]:
# 2 - 확인
new_log

[{'recordId': 54329,
  'ArrivalTimeStamp': 1679051522.413,
  'data': {'user_id': 'DJj5XF2IcJ9OUhpzfwr0vgygyH5rihXz_h5DjEK65_Uw',
   'record_id': 54329,
   'activity': 'cart',
   'url': '/api/products/product/',
   'method': 'POST',
   'name': 'json_logger',
   'inDate': '2023-03-17T11:12:02.413Z',
   'detail': {'message': 'POST cart', 'levelname': 'INFO'}}},
 {'recordId': 54330,
  'ArrivalTimeStamp': 1679051522.7,
  'data': {'user_id': 'M2Eg8QukLp6LXl5yXA9JGwynzRlowxj0PY7Hwpqh-ZCg',
   'record_id': 54330,
   'activity': 'purchase',
   'url': '/api/products/product/',
   'method': 'POST',
   'name': 'json_logger',
   'inDate': '2023-03-17T11:12:02.700Z',
   'detail': {'message': 'POST purchase', 'levelname': 'INFO'}}},
 {'recordId': 54331,
  'ArrivalTimeStamp': 1679051522.988,
  'data': {'user_id': 'tEr7598vVP81J87dNHbW0gRUzADi-FquJftwEQ4js_Pg',
   'record_id': 54331,
   'activity': 'view',
   'url': '/api/products/product/',
   'method': 'POST',
   'name': 'json_logger',
   'inDate': '

In [183]:
# 3. 'method' 를 value 별 숫자로 압축
# 'method' 의 value 별 숫자를 저장한 dict를 사용
# 새로운 method 가 등장할 때 마다, 변환 후 업데이트
import value_comp
import importlib        # dict 비교를 위한 모듈(모듈 재호출 기능)
VALUE_DICT_PATH = './value_comp.py'

def comp_method(log, dict):
    for i in log:
        item = i['data']['method']
        # 호출된 dict 에 값이 없는 경우(새로운 method)
        if item not in dict:
            # dict 가 비어있으면 새로운 값에 1 할당
            if dict == {}:
                dict[item] = 1
            # dict 에 다른 값이 있으면 새로운 값에 마지막 숫자 + 1 할당
            else:
                dict[item] = len(dict) + 1
            i['data']['method'] = dict[item]
        # 호출된 dict 에 값이 있는 경우
        else:
            i['data']['method'] = dict[item]
    return

# method value 압축
comp_method(new_log, value_comp.method_dict)
# 압축 후 (업데이트된) method_dict
new_method_dict = value_comp.method_dict

# 업데이트전(수정되지 않은 dict)의 원본 method_dict 호출
importlib.reload(value_comp)
old_method_dict = value_comp.method_dict

# 원본 method_dict 와 압축 후 method_dict를 비교
# 업데이트된 값이 있으면 원본 method_dict를 업데이트
if old_method_dict != new_method_dict:
    with open(VALUE_DICT_PATH, 'r') as f:
        lines = f.readlines()
    # line[0] 의 method_dict 만 수정
    lines[0] = f'method_dict = {new_method_dict}\n' 

    with open(VALUE_DICT_PATH, 'w') as f:
        f.writelines(lines)

In [184]:
# 3 - 확인
new_log

[{'recordId': 54329,
  'ArrivalTimeStamp': 1679051522.413,
  'data': {'user_id': 'DJj5XF2IcJ9OUhpzfwr0vgygyH5rihXz_h5DjEK65_Uw',
   'record_id': 54329,
   'activity': 'cart',
   'url': '/api/products/product/',
   'method': 1,
   'name': 'json_logger',
   'inDate': '2023-03-17T11:12:02.413Z',
   'detail': {'message': 'POST cart', 'levelname': 'INFO'}}},
 {'recordId': 54330,
  'ArrivalTimeStamp': 1679051522.7,
  'data': {'user_id': 'M2Eg8QukLp6LXl5yXA9JGwynzRlowxj0PY7Hwpqh-ZCg',
   'record_id': 54330,
   'activity': 'purchase',
   'url': '/api/products/product/',
   'method': 1,
   'name': 'json_logger',
   'inDate': '2023-03-17T11:12:02.700Z',
   'detail': {'message': 'POST purchase', 'levelname': 'INFO'}}},
 {'recordId': 54331,
  'ArrivalTimeStamp': 1679051522.988,
  'data': {'user_id': 'tEr7598vVP81J87dNHbW0gRUzADi-FquJftwEQ4js_Pg',
   'record_id': 54331,
   'activity': 'view',
   'url': '/api/products/product/',
   'method': 1,
   'name': 'json_logger',
   'inDate': '2023-03-17T11:1

In [185]:
# 4. 'url' 을 value 별 숫자로 압축
# '/api/products/product/' 를 base로, 다음에 붙는 숫자를 value 로 압축
# base 만 있으면(숫자가 없으면) base : 0 할당
# 'method' 압축과 마찬가지로 진행
def comp_url(log, dict):
    for i in log:
        item = i['data']['url']
        # 호출된 dict 에 값이 없는 경우(새로운 url)
        if item not in dict:
            # base 만 존재하면, 0 할당
            if len(item) == 22:
                dict[item] = 0
            # base + 숫자 구성 이면, 해당 숫자 할당
            else:
                dict[item] = int(item[22:])
            i['data']['url'] = dict[item]
        # 호출된 dict 에 값이 있는 경우
        else:
            i['data']['url'] = dict[item]
    return
        
# url value 압축
comp_url(new_log, value_comp.url_dict)

# 압축 후 (업데이트된) url_dict
new_url_dict = value_comp.url_dict

# 업데이트전(수정되지 않은 dict)의 원본 url_dict 호출
importlib.reload(value_comp)
old_url_dict = value_comp.url_dict

# 원본 url_dict 와 압축 후 url_dict를 비교
# 업데이트된 값이 있으면 원본 url_dict를 업데이트
if old_url_dict != new_url_dict:
    with open(VALUE_DICT_PATH, 'r') as f:
        lines = f.readlines()
    # line[1] 의 url_dict 만 수정
    lines[1] = f'url_dict = {new_url_dict}\n' 

    with open(VALUE_DICT_PATH, 'w') as f:
        f.writelines(lines)

In [186]:
# 4 - 확인
new_log

[{'recordId': 54329,
  'ArrivalTimeStamp': 1679051522.413,
  'data': {'user_id': 'DJj5XF2IcJ9OUhpzfwr0vgygyH5rihXz_h5DjEK65_Uw',
   'record_id': 54329,
   'activity': 'cart',
   'url': 0,
   'method': 1,
   'name': 'json_logger',
   'inDate': '2023-03-17T11:12:02.413Z',
   'detail': {'message': 'POST cart', 'levelname': 'INFO'}}},
 {'recordId': 54330,
  'ArrivalTimeStamp': 1679051522.7,
  'data': {'user_id': 'M2Eg8QukLp6LXl5yXA9JGwynzRlowxj0PY7Hwpqh-ZCg',
   'record_id': 54330,
   'activity': 'purchase',
   'url': 0,
   'method': 1,
   'name': 'json_logger',
   'inDate': '2023-03-17T11:12:02.700Z',
   'detail': {'message': 'POST purchase', 'levelname': 'INFO'}}},
 {'recordId': 54331,
  'ArrivalTimeStamp': 1679051522.988,
  'data': {'user_id': 'tEr7598vVP81J87dNHbW0gRUzADi-FquJftwEQ4js_Pg',
   'record_id': 54331,
   'activity': 'view',
   'url': 0,
   'method': 1,
   'name': 'json_logger',
   'inDate': '2023-03-17T11:12:02.988Z',
   'detail': {'message': 'POST view', 'levelname': 'INFO'

In [187]:
# 5. 'inDate' 다른 표기방식으로 압축 / 'inDate' key 추가
from datetime import datetime

def trans_indate_plus(data):
    for i in data:
        inDate = i['data']['inDate']
        date = datetime.fromisoformat(inDate[:-1])
        outDate = date.strftime('%y%m%d%H%M%S%f')
        i['data']['inDate'] = outDate
        i['inDate'] = outDate

trans_indate_plus(new_log)

In [188]:
# 5 - 확인
new_log

[{'recordId': 54329,
  'ArrivalTimeStamp': 1679051522.413,
  'data': {'user_id': 'DJj5XF2IcJ9OUhpzfwr0vgygyH5rihXz_h5DjEK65_Uw',
   'record_id': 54329,
   'activity': 'cart',
   'url': 0,
   'method': 1,
   'name': 'json_logger',
   'inDate': '230317111202413000',
   'detail': {'message': 'POST cart', 'levelname': 'INFO'}},
  'inDate': '230317111202413000'},
 {'recordId': 54330,
  'ArrivalTimeStamp': 1679051522.7,
  'data': {'user_id': 'M2Eg8QukLp6LXl5yXA9JGwynzRlowxj0PY7Hwpqh-ZCg',
   'record_id': 54330,
   'activity': 'purchase',
   'url': 0,
   'method': 1,
   'name': 'json_logger',
   'inDate': '230317111202700000',
   'detail': {'message': 'POST purchase', 'levelname': 'INFO'}},
  'inDate': '230317111202700000'},
 {'recordId': 54331,
  'ArrivalTimeStamp': 1679051522.988,
  'data': {'user_id': 'tEr7598vVP81J87dNHbW0gRUzADi-FquJftwEQ4js_Pg',
   'record_id': 54331,
   'activity': 'view',
   'url': 0,
   'method': 1,
   'name': 'json_logger',
   'inDate': '230317111202988000',
   'det

In [189]:
# 6. 암호화
def encrypt_data(log):
    for i in log:
        json_log = i['data']
        fernet = Fernet(key)
        encrypt_str = fernet.encrypt(f'{json_log}'.encode('ascii'))
        i['data'] = encrypt_str.decode()

encrypt_data(new_log)

In [190]:
# 6 - 확인
new_log

[{'recordId': 54329,
  'ArrivalTimeStamp': 1679051522.413,
  'data': 'gAAAAABkE8yYQ-CVurvMkBjKwdHhlQ8JfJQSRAZGMIYAC7JWJB2tKFkSkz98rZ5aiQ2ooZjJ6BBRPqIYSeKS8cuxadNUty5cIhg0m5UXWoVmcmNPFarS_KMe4iYVeCW5H7qtCgI2BNroQEe3ITANQ8j56TbLcVxaOG5yjH3nZQ3rkoVF6se2JzonSyJJn-R5oQzFiqIAn6ghpN7GpfXV729VJPnAQYmWOvajrydPx_HQ8G4A_XdtKL2sAfBVVGAemDEdj_43ppieFkrzuRGi48AeWuUxDI54N-eYexWfsrZk25lZd9lj0K20yZLukKQxzRAhZhEbxmwXfZBc7N3CBus8tBqR23str6OuEpdZrr35ZUbfrqeeVy_h0sVC5rGHPe3U1uRBTimH',
  'inDate': '230317111202413000'},
 {'recordId': 54330,
  'ArrivalTimeStamp': 1679051522.7,
  'data': 'gAAAAABkE8yYOG8W2W0m0gHAS3QGMTGBDAuXLv4sLsjl0J1XbHk2Dz6vk9IQwVjTHBlP94UivOnmc2o1Bgze9gztG4-cZRnoDHGSDWO1TP6jNqOlq5dWKlzJJFiqRt398mar6YAZNUF4G6Rj49zjZ7B8OHTBfGuoxQT2N4jQ0GbZklBN3agL2LaZPotsT6NWi4wA-yHqPTkDsbBdMU5BeiRZ0H1LgK2UdF9ejuBgAnIWN18xMBBiShjHqDufQ-q3FFs51Tg44kUuFY8qUNoRFVN2QQC3aRImAGt_ga3ZNqPxiqiGBy98k87kMN3Te8maBuIJjt_jS5y4rCrYZMtzoBPgngniksAXZuay_bOwt0Vct5ksjz9ZWxsY3SzWAMAU4IQciI_OxfOEZmhN6DjYOETh5EgtQijZdw==',
  'in

In [191]:
# 7. 압축 처리한 log, 새로운 파일에 저장(.json)
# 중복된 recordId 는 앞에서 처리 했기 때문에, 중복체크 필요없음
COMP_LOG_PATH_NOTE = './log_note/comp_log_practice.json'

def save_comp_log(filepath, log):
    # 의미없는 key 삭제
    for i in log:
        i.pop('recordId', None)
        i.pop('ArrivalTimeStamp', None)
    # path 를 가진 파일(로그가 저장된 파일)이 없을 경우, log 저장할 파일 생성
    if not os.path.isfile(filepath):
        with open(filepath, 'w') as f:
            json.dump(log, f, indent = 2)
    # path 를 가진 파일 있을 경우, 해당 파일에 이어서 log 저장
    else:
        with open(filepath, 'r') as f:
            log_exist = json.load(f)
            log_exist.extend(log)
        with open(filepath, 'w') as f:
            json.dump(log_exist, f, indent = 2)
    return log

comp_log = save_comp_log(COMP_LOG_PATH_NOTE, new_log)

In [192]:
# 7 - 확인
comp_log

[{'data': 'gAAAAABkE8yYQ-CVurvMkBjKwdHhlQ8JfJQSRAZGMIYAC7JWJB2tKFkSkz98rZ5aiQ2ooZjJ6BBRPqIYSeKS8cuxadNUty5cIhg0m5UXWoVmcmNPFarS_KMe4iYVeCW5H7qtCgI2BNroQEe3ITANQ8j56TbLcVxaOG5yjH3nZQ3rkoVF6se2JzonSyJJn-R5oQzFiqIAn6ghpN7GpfXV729VJPnAQYmWOvajrydPx_HQ8G4A_XdtKL2sAfBVVGAemDEdj_43ppieFkrzuRGi48AeWuUxDI54N-eYexWfsrZk25lZd9lj0K20yZLukKQxzRAhZhEbxmwXfZBc7N3CBus8tBqR23str6OuEpdZrr35ZUbfrqeeVy_h0sVC5rGHPe3U1uRBTimH',
  'inDate': '230317111202413000'},
 {'data': 'gAAAAABkE8yYOG8W2W0m0gHAS3QGMTGBDAuXLv4sLsjl0J1XbHk2Dz6vk9IQwVjTHBlP94UivOnmc2o1Bgze9gztG4-cZRnoDHGSDWO1TP6jNqOlq5dWKlzJJFiqRt398mar6YAZNUF4G6Rj49zjZ7B8OHTBfGuoxQT2N4jQ0GbZklBN3agL2LaZPotsT6NWi4wA-yHqPTkDsbBdMU5BeiRZ0H1LgK2UdF9ejuBgAnIWN18xMBBiShjHqDufQ-q3FFs51Tg44kUuFY8qUNoRFVN2QQC3aRImAGt_ga3ZNqPxiqiGBy98k87kMN3Te8maBuIJjt_jS5y4rCrYZMtzoBPgngniksAXZuay_bOwt0Vct5ksjz9ZWxsY3SzWAMAU4IQciI_OxfOEZmhN6DjYOETh5EgtQijZdw==',
  'inDate': '230317111202700000'},
 {'data': 'gAAAAABkE8yYG6pyfhTn5gtOA9frnrVAESQqU22Oe8o-mCdNfZtTkfkrdXEArtiYBbNjBDrZ-Zk